# Flask请求方式，Session, Cookie
## request 请求

- args--->  GET请求，获取参数

- form--->  POST请求，获取参数

- files-->  上传的file文件

- method--> 请求方式

##  蓝图前缀

- url_prefix=‘hello’
```python
app.register_blueprint(blueprint=blue, url_prefix='/hello')
```

## response 响应

- 服务端自己创建，然后返回给客户端

- 传入自己定义的状态码

- make_response(‘页面’，状态码)




In [ ]:
from flask import render_template, request, make_response, redirect


from flask import Blueprint, url_for, abort


@blue.route('/getrequest/', methods=['GET', 'POST'])
def get_request():
    if request.method == 'GET':
        args = request.args
    else:
        form = request.form
    return '获取request'


# 返回响应
@blue.route('/makeresponse/')
def makeResponse():
    # 获取一个响应
    # response = make_response('<h3>sjkojasklf</h3>')
    temp = render_template('hello.html')
    # 返回页面
    response = make_response(temp)
    # response = make_response(temp, 300)
    # 300表示告诉flask状态码 ,两种方式效果是一样的

    return response, 300


# 跳转
@blue.route('/redirect/')
def makeRedirect():
    # 第一种方法
    # return redirect('/hello/index/')
    # 第二种方法,定义的蓝图名称加方法名，注意不是url名
    return redirect(url_for('first.makeResponse'))


# 抛出异常
@blue.route('/makeabort/')
def make_abort():
    abort(400)
    return '终结'


# 捕捉异常
@blue.errorhandler(400)
def get_error(exception):
    return '捕捉异常：%s' % exception



## session
- 这里的session关联redis非关系性数据库，它的主要作用是储存一些重要数据，例如下面的登录方法中，得到session中的用户名，这样可以随时保存当前用户。

- 安装:  
    - pip install flask-session
    - pip install redis
- ubuntu 安装redis
    - apt-get install redis-server 

- 使用数据库：

	- SESSION_TYPE类型
	- redis
	- mongodb
	- memcached
	- sqlchemy

In [ ]:
#  __init__中配置信息
import os

from flask import Flask
from flask_session import Session
import redis
from App.views import blue



def create_app():

    # 定义初始化路径
    BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    # 指定templates文件夹路径
    templates_dir = os.path.join(BASE_DIR, 'templates')
    # 指定静态文件夹路径
    static_dir = os.path.join(BASE_DIR, 'static')

    # 初始化， __name__代表主模块名或者包
    app = Flask(__name__, static_folder=static_dir, template_folder=templates_dir)
    # url_prefix加前缀，用于区分不同的app
    app.register_blueprint(blueprint=blue, url_prefix='/hello')
    # 密钥，必须要写
    app.config['SECRET_KEY'] = 'secret_key'
    # 使用redis存储信息,默认访问redis了， 127.0.01:6379
    app.config['SESSION_TYPE'] = 'redis'
    # 访问redis, 可以访问远程数据库
    #app.config['SESSION_REDIS'] = redis.Redis(host='47.105.32.183', port='6379')
    # 修改redis中key的前缀
    app.config['SESSION_KEY_PREFIX'] = 'flask'

    #  初始化app第一种方式
    Session(app)

    # 第二种
    # se = Session()
    # se.init_app(app)
    return app
    

In [ ]:
# views.py
import random
from flask import Blueprint, render_template, request, session, redirect, url_for, make_response


blue = Blueprint('app', __name__)


# 登录页面
@blue.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'GET':
        # 从redis中取到当前的username
        username = session.get('username')
        return render_template('login_new.html', username=username)
    else:
        # 接收post请求，并从中提取username
        username = request.form.get('username')
        # 将username储存到redis中
        session['username'] = username

        return redirect(url_for('app.login'))


# 创建cookie
@blue.route('/getresponse')
def get_response():

    response = make_response('<h2>呵呵</h2>', 200)
    ticket = ''
    s = 'qweriopasdfghjklzxcvbnm'
    for i in range(20):
        ticket += random.choice(s)
    # max_age 以秒为单位的过期时间 ，expire 以天为单位的过期时间
    response.set_cookie('ticket', ticket)
    return response


# 删除cookie
@blue.route('/deletecookie')
def del_cookie():
    response = make_response('<h2>as</h2>')
    response.delete_cookie('ticket')
    return response